In [ ]:
!pip install sympy numpy pandas matplotlib scipy ipython plotly nbformat
# this should take up to 7 minutes

In [3]:
import warnings
# Ignore SymPyDeprecationWarning
warnings.filterwarnings("ignore")
from sympy import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fsolve
from IPython.display import display, Math, Latex
from plotly import graph_objects as go

In [4]:

def model_simulator(y, t, model_dictionary):
    for time, cur_model in model_dictionary.items():
        if time[0] <= t < time[1]:
            return cur_model['model'](*y).flatten()


def model_fixed_point_function(ic, model_function):
    return model_function(*ic).flatten()


def simulate_HPA(ic, sim_time, model_dictionary, variables):
    result = odeint(model_simulator, ic, sim_time, args=(model_dictionary,), hmax=0.01)
    result_df = pd.DataFrame(result, columns=variables)
    return result_df

In [ ]:
def crate_symbol(name):
    return symbols(name)  # , real=True, positive=True)


u = crate_symbol("u")

x1 = crate_symbol("x1")
x2 = crate_symbol("x2")
x3 = crate_symbol("x3")
x4 = crate_symbol("x4")

a1 = crate_symbol("a1")
a2 = crate_symbol("a2")
a3 = crate_symbol("a3")
a4 = crate_symbol("a4")
a5 = crate_symbol("a5")

b1 = crate_symbol("b1")
b2 = crate_symbol("b2")
b3 = crate_symbol("b3")
b4 = crate_symbol("b4")
b5 = crate_symbol("b5")

K_GR = crate_symbol("K_GR")
H1 = crate_symbol("H1")
H2 = crate_symbol("H2")
P = crate_symbol("P")
A = crate_symbol("A")
t = crate_symbol("t")

aP = crate_symbol("aP")
aA = crate_symbol("aA")

bP = crate_symbol("bP")
bA = crate_symbol("bA")

mr = 1 / x3
gr = K_GR**3 / (x3**3 + K_GR**3)
gr = 1/(1+(x3/K_GR)**3)
dx1_dt = b1 * u * gr * mr - a1 * x1
dx2_dt = b2 * x1 * P * gr - a2 * x2
dx3_dt = b3 * x2 * A * x4 * gr - a3 * x3
dx4_dt = b4 * u * gr  - a4 * x4
dP_dt = P * (bP * x1 - aP)
dA_dt = A * (bA * x2 - aA)

display(
    Matrix(
        [
            Eq(Derivative(x1, t), dx1_dt),
            Eq(Derivative(x2, t), dx2_dt),
            Eq(Derivative(x3, t), dx3_dt),
            Eq(Derivative(x4, t), dx4_dt),
            Eq(Derivative(P, t), dP_dt),
            Eq(Derivative(A, t), dA_dt),
        ]
    )
)
# ignore the warning
model = Matrix([dx1_dt, dx2_dt, dx3_dt, dx4_dt, dP_dt, dA_dt])
variables = (x1, x2, x3, x4, P, A)

C:\Users\qywoe\AppData\Local\Temp\ipykernel_39480\3082687162.py:48: SymPyDeprecationWarning:



non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.




Matrix([
[Eq(Derivative(x1, t), -a1*x1 + b1*u/(x3*(1 + x3**3/K_GR**3)))],
[   Eq(Derivative(x2, t), P*b2*x1/(1 + x3**3/K_GR**3) - a2*x2)],
[Eq(Derivative(x3, t), A*b3*x2*x4/(1 + x3**3/K_GR**3) - a3*x3)],
[Eq(Derivative(x4, t), -a4*x4 + b4*u/(x3*(1 + x3**3/K_GR**3)))],
[                        Eq(Derivative(P, t), P*(-aP + bP*x1))],
[                        Eq(Derivative(A, t), A*(-aA + bA*x2))]])

In [57]:
Matrix(solve(dx4_dt,x4))

Matrix([[K_GR**3*b4*u/(a4*(K_GR**3 + x3**3))]])

In [30]:
parameters = {
    u: 1,
    a1: 1,
    a2: 1,
    a3: 1,
    a4: 1,
    a5: 1,
    b1: 1,
    b2: 1,
    b3: 1,
    b4: 1,
    b5: 1,
    K_GR: 4,
    H1: 1,
    H2: 1,
    aA: 1,
    aP: 1,
    bA: 1,
    bP: 1,
    # P: 1,
    # A: 1,
    # x1: 1,
    # x2: 1,
    # x3: 1,
    # x4: 1,
    # x5: 1,
}

parameter_values_normal = {
    u: 1,
    b1: 0.17 * 60 * 24,
    b2: 0.035 * 60 * 24 * 2,
    b3: 0.0086 * 60 * 24,
    b4: 0.0086 * 60 * 24,
    b5: 0.0086 * 60 * 24,
    bP: 0.049,
    bA: 0.099,
    K_GR: 6,
    a1: 0.17 * 60 * 24,
    a2: 0.035 * 60 * 24 * 2,
    a3: 0.0086 * 60 * 24,
    a4: 0.0086 * 60 * 24,
    a5: 0.0086 * 60 * 24,
    aP: 0.049,
    aA: 0.099,
} 

In [60]:
parameter_values_sim_high = parameter_values_normal.copy()
parameter_values_sim_low = parameter_values_normal.copy()
parameter_values_sim_high[u] = 2

model_high_u = lambdify(variables, model.subs(parameter_values_sim_high))
model_low_u = lambdify(variables, model.subs(parameter_values_sim_low))
parameter_values_sim_high[K_GR] = 1
model_high_u_low_kgr = lambdify(variables, model.subs(parameter_values_sim_high))
parameter_values_sim_low[K_GR] = 1
model_low_u_low_kgr = lambdify(variables, model.subs(parameter_values_sim_low))

guess = [1 for _ in range(len(variables))]

low = {"model": model_low_u, "params": parameter_values_sim_low}
high = {"model": model_high_u, "params": parameter_values_sim_high}
low_kgr = {"model": model_low_u_low_kgr, "params": parameter_values_sim_low}
high_kgr = {"model": model_high_u_low_kgr, "params": parameter_values_sim_high}

day = 1
hour = day / 24
year = 365 * day
L = 300 * day
dt = hour
sim_time = np.arange(0, L, dt)

model_dictionary = {  # specify which model to use in each time interval
    (5 * day, 8 * day): high,
    (8 * day, 10 * day): low,
    (10 * day, 30 * day): low_kgr,
    (88 * day, 91 * day): high_kgr,
    (30 * day, np.inf * day): low_kgr,
    (-np.inf, np.inf): low,
}

ic = fsolve(model_fixed_point_function, guess, args=model_low_u)

res = simulate_HPA(ic, sim_time, model_dictionary, variables)
time_scale = "days"
if time_scale == "hours":
    res["hours"] = sim_time * 24
    res.set_index("hours", inplace=True)
else:
    res["days"] = sim_time
    res.set_index("days", inplace=True)
    
res.plot(backend="plotly").show()
plt.tight_layout()
# plt.xticks(np.arange(0, L, day), np.arange(0, L, 2 * day))
plt.show()

<Figure size 640x480 with 0 Axes>

In [18]:
def calculate_max_min_ratio(res, start_time, end_time):
    # Filter the DataFrame for the given time interval
    interval_data = res.loc[start_time:end_time]

    # Get the max and min values for the specified column
    results = {}

    # Iterate over each column in the DataFrame
    for column in interval_data.columns:
        max_value = interval_data[column].max()
        min_value = interval_data[column].min()
        ratio = max_value / min_value if min_value != 0 else float("inf")

        # Store the results in the dictionary
        results[column] = {"max": max_value, "min": min_value, "ratio": ratio}
    # Print the results in a nice format
    return results


def print_results(results):
    for col, stats in results.items():
        print(f"Column: {col}")
        print(f"  Max: {stats['max']}")
        print(f"  Min: {stats['min']}")
        print(f"  Max/Min Ratio: {stats['ratio']}")
        print()


# Example usage
start_time = 0 * day  # specify the start time
end_time = 10 * day  # specify the end time
result = calculate_max_min_ratio(res, start_time, end_time)
# Check if the ratio of x4 in the first 10 days is smaller than the ratio in days 30 to 40
start_time_30 = 30 * day
end_time_40 = 40 * day

result_30_40 = calculate_max_min_ratio(res, start_time_30, end_time_40)

print(f'the ratio of the first 10 is {result[x4]["ratio"]} and the ratio of the 30 to 40 is {result_30_40[x4]["ratio"]}')
# print(result[x4])
# print_results(result)

the ratio of the first 10 is 1.9309904621365366 and the ratio of the 30 to 40 is 1.8268524690666805
